In [1]:
from current_season_code.functions import *
import datetime as dt

In [7]:
formatted_data = format_api_data(2021)
columns = list(formatted_data.columns)
data = formatted_data[['Date_y']+columns[:-7]+columns[-6:]]
columns = ['Date', 'GAME_ID', 'TEAM_NAME', 'TEAM_NAME_Opp', 'FGM', 'FGA',
       'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA',
       'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK',
       'TO', 'PF', 'PTS', 'PLUS_MINUS', 'E_OFF_RATING',
       'OFF_RATING', 'E_DEF_RATING', 'DEF_RATING', 'E_NET_RATING',
       'NET_RATING', 'AST_PCT', 'AST_TOV', 'AST_RATIO', 'OREB_PCT',
       'DREB_PCT', 'REB_PCT', 'E_TM_TOV_PCT', 'TM_TOV_PCT',
       'EFG_PCT', 'TS_PCT', 'USG_PCT', 'E_USG_PCT', 'E_PACE',
       'PACE', 'PACE_PER40', 'POSS', 'PIE', 'drop_1', 'drop_2',
       'drop_3', 'FGM_Opp', 'FGA_Opp', 'FG_PCT_Opp', 'FG3M_Opp', 'FG3A_Opp',
       'FG3_PCT_Opp', 'FTM_Opp', 'FTA_Opp', 'FT_PCT_Opp', 'OREB_Opp', 'DREB_Opp', 'REB_Opp',
       'AST_Opp', 'STL_Opp', 'BLK_Opp', 'TO_Opp', 'PF_Opp', 'PTS_Opp', 'PLUS_MINUS_Opp',
       'E_OFF_RATING_Opp', 'OFF_RATING_Opp', 'E_DEF_RATING_Opp', 'DEF_RATING_Opp',
       'E_NET_RATING_Opp', 'NET_RATING_Opp', 'AST_PCT_Opp', 'AST_TOV_Opp',
       'AST_RATIO_Opp', 'OREB_PCT_Opp', 'DREB_PCT_Opp', 'REB_PCT_Opp',
       'E_TM_TOV_PCT_Opp', 'TM_TOV_PCT_Opp', 'EFG_PCT_Opp', 'TS_PCT_Opp', 'USG_PCT_Opp',
       'E_USG_PCT_Opp', 'E_PACE_Opp', 'PACE_Opp', 'PACE_PER40_Opp', 'POSS_Opp', 'PIE_Opp',
       'Wins', 'Losses', 'Wins_Opp', 'Losses_Opp',
       'Win_Pct', 'Win_Pct_Opp']
data.columns = columns
data[:50]

(96, 97)


,Date,GAME_ID,TEAM_NAME,TEAM_NAME_Opp,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,...,PACE_Opp,PACE_PER40_Opp,POSS_Opp,PIE_Opp,Wins,Losses,Wins_Opp,Losses_Opp,Win_Pct,Win_Pct_Opp
0,2021-10-19,22100001,Bucks,Nets,48,105,0.457,17,45,0.378,...,102.00,85.00,102.0,0.407,0.0,0.0,0.0,0.0,NaN,NaN
1,2021-10-19,22100002,Lakers,Warriors,45,95,0.474,15,42,0.357,...,112.50,93.75,113.0,0.578,0.0,0.0,0.0,0.0,NaN,NaN
2,2021-10-20,22100003,Pacers,Hornets,42,90,0.467,17,47,0.362,...,106.50,88.75,106.0,0.480,0.0,0.0,0.0,0.0,NaN,NaN
3,2021-10-20,22100004,Pistons,Bulls,36,90,0.400,6,28,0.214,...,98.00,81.67,98.0,0.545,0.0,0.0,0.0,0.0,NaN,NaN
4,2021-10-20,22100005,Knicks,Celtics,51,105,0.486,17,45,0.378,...,103.03,85.86,124.0,0.479,0.0,0.0,0.0,0.0,NaN,NaN
5,2021-10-20,22100006,Wizards,Raptors,36,81,0.444,5,27,0.185,...,106.50,88.75,107.0,0.300,0.0,0.0,0.0,0.0,NaN,NaN
6,2021-10-20,22100007,Grizzlies,Cavaliers,53,100,0.530,14,33,0.424,...,104.50,87.08,104.0,0.456,0.0,0.0,0.0,0.0,NaN,NaN
7,2021-10-20,22100008,Timberwolves,Rockets,42,87,0.483,16,38,0.421,...,108.00,90.00,108.0,0.373,0.0,0.0,0.0,0.0,NaN,NaN
8,2021-10-20,22100009,76ers,Pelicans,44,85,0.518,13,25,0.520,...,99.00,82.50,99.0,0.400,0.0,0.0,0.0,0.0,NaN,NaN
9,2021-10-20,22100010,Spurs,Magic,49,98,0.500,13,30,0.433,...,95.50,79.58,95.0,0.371,0.0,0.0,0.0,0.0,NaN,NaN


In [17]:
formatted_data['is_B2B'] = 0
formatted_data['is_B2B_Opp'] = 0
formatted_data['is_B2B_First'] = 0
formatted_data['is_B2B_First_Opp'] = 0
formatted_data['is_B2B_Second'] = 0
formatted_data['is_B2B_Second_Opp'] = 0
formatted_data['Days_Rest_Team'] = 0
formatted_data['Days_Next Game'] = 0
formatted_data['Days_Rest_Team_Opp'] = 0
formatted_data['Days_Next Game_Opp'] = 0
formatted_data['is_Home'] = 0
formatted_data['is_Home_Opp'] = 0

In [22]:
dt.date.today() - dt.timedelta(days = 1)

datetime.date(2021, 10, 22)

In [32]:
pd.read_csv('current_season_data/api_data_2021_advanced.csv')

FileNotFoundError: [Errno 2] File current_season_data/api_data_2021_advanced.csv does not exist: 'current_season_data/api_data_2021_advanced.csv'

In [11]:
odds = pd.read_csv('current_season_data/yesterday_odds.csv', index_col = 0)

In [12]:
odds

,Home_Team,Away_Team,Home_Odds,Away_Odds,Date,Home_Prob,Away_Prob
0,DAL,SAC,-180.0,152.0,2021-10-31,64.285714,39.682540
1,MIL,UTA,110.0,-130.0,2021-10-31,47.619048,56.521739
2,CHA,POR,-116.0,-102.0,2021-10-31,53.703704,50.495050
3,BKN,DET,-800.0,560.0,2021-10-31,88.888889,15.151515
4,LAL,HOU,-750.0,530.0,2021-10-31,88.235294,15.873016
